In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import pickle
import joblib

torch.manual_seed(42)
np.random.seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
DATA_PATH = 'C:/Users/shrey/Downloads/EV Charging Station Dynamic Pricing-20250701T181708Z-1-001/EV Charging Station Dynamic Pricing/data/urban_ev_combined.csv'
ADJ_PATH = 'C:/Users/shrey/Downloads/EV Charging Station Dynamic Pricing-20250701T181708Z-1-001/EV Charging Station Dynamic Pricing/data/adj_filter.csv'
MODEL_SAVE_DIR = 'C:/Users/shrey/Downloads/EV Charging Station Dynamic Pricing-20250701T181708Z-1-001/EV Charging Station Dynamic Pricing/models'
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

SEQ_LEN = 72 # Taking 72 hours data
EPOCHS = 25
BATCH_SIZE = 64
HIDDEN_SIZE = 128
PATIENCE = 5
LEARNING_RATE = 0.001

df = pd.read_csv(DATA_PATH, parse_dates=['time'])
adj = pd.read_csv(ADJ_PATH, index_col=0)
adj.columns = adj.columns.astype(str)
adj.index = adj.index.astype(str)

# Only using the selected features based on the importance
features = [
    'is_peak', 'duration', 'duration_24h_mean',
    'charge_count', 'occupancy', 'e_price', 's_price', 'hour', 'area'
]

target = 'volume'
df.dropna(subset=features + [target], inplace=True)

df['power_kw'] = df['volume'] / df['duration']
df['power_kw'] = df['power_kw'].replace([np.inf, -np.inf], np.nan).fillna(0)
features.append('power_kw')

df[target] = np.log1p(df[target].clip(lower=0))  # Applying log transform and clipping
for col in features:
    if df[col].dtype in [np.float64, np.float32]:
        mean = df[col].mean()
        std = df[col].std()
        df[col] = df[col].clip(lower=mean - 3*std, upper=mean + 3*std)


scalers = {}
for col in features + [target]:
    flat = df[col].astype(np.float32).values.reshape(-1, 1)
    if np.any(np.isnan(flat)):
        raise ValueError(f"Column {col} contains NaNs")
    if np.var(flat) == 0:
        raise ValueError(f"Column {col} has zero variance")
        flat += np.random.normal(0, 1e-6, flat.shape) # Adding a small noise in case of zero variance

    scaler = StandardScaler()
    df[col] = scaler.fit_transform(flat).flatten()
    scalers[col] = scaler


In [9]:
class LSTM_Model(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=HIDDEN_SIZE, num_layers=2, batch_first=True, dropout=0.3)
        self.norm = nn.LayerNorm(HIDDEN_SIZE)
        self.fc = nn.Sequential(
            nn.Linear(HIDDEN_SIZE, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.norm(out[:, -1, :])
        return self.fc(out)

In [10]:
class ZoneDataset(Dataset):
    def __init__(self, df_zone, features, target, seq_len):
        self.X, self.y = [], []
        for i in range(len(df_zone) - seq_len):
            x_seq = df_zone[features].iloc[i:i + seq_len].values
            x_seq = np.array(x_seq, dtype=np.float32)

            y_val = df_zone[target].iloc[i + seq_len - 1]
            y_val = np.array(y_val, dtype=np.float32).flatten()

            self.X.append(x_seq)
            self.y.append(y_val)

        self.X = torch.tensor(self.X, dtype=torch.float32)
        self.y = torch.tensor(self.y, dtype=torch.float32)

        if self.y.dim() == 1:
            self.y = self.y.view(-1, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [11]:
def add_neighbour_avg(df, zone_id, target_col='volume_24h_mean'): # Neighbour volume averaging
    neighbours = adj.loc[str(zone_id)][adj.loc[str(zone_id)] == 1].index.tolist()
    df_main = df[df['zone_id'] == zone_id][['time', target_col]].set_index('time')
    if not neighbours:
        df_main[f'neighbour_{target_col}'] = 0
    else:
        neighbour_data = df[df['zone_id'].isin(neighbours)][['time', target_col]]
        neighbour_mean = (
            neighbour_data.groupby('time')[target_col]
            .mean()
            .reindex(df_main.index)
            .fillna(0)
            .rename(f'neighbour_{target_col}')
        )
        df_main[f'neighbour_{target_col}'] = neighbour_mean

    df_out = df[df['zone_id'] == zone_id].copy()
    df_out[f'neighbour_{target_col}'] = df_main[f'neighbour_{target_col}'].values
    return df_out


In [16]:
logs = []
zone_ids = df['zone_id'].unique()

for zone_id in tqdm(zone_ids, desc="Training per zone"):
    try:
        # Compute neighbor averages
        df_zone = add_neighbour_avg(df, zone_id, target_col='volume')  # Use 'volume' instead of 'volume_24h_mean'
        df_zone = add_neighbour_avg(df_zone, zone_id, target_col='duration_24h_mean')
        df_zone.dropna(subset=features + [target, 'neighbour_volume', 'neighbour_duration_24h_mean'], inplace=True)
        all_features = features + ['neighbour_volume', 'neighbour_duration_24h_mean']

        # Check for low target variance
        if df_zone[target].var() < 1e-6:
            print(f"Skipping zone {zone_id}: target variance too low ({df_zone[target].var()})")
            continue

        # Select features with non-zero variance
        valid_features = []
        for col in all_features:
            flat = df_zone[col].astype(np.float32).values
            if np.var(flat) > 1e-6:
                valid_features.append(col)

        if not valid_features:
            print(f"Skipping zone {zone_id}: no valid features with non-zero variance")
            continue

        # Scale zone-specific data
        zone_scalers = {}
        for col in valid_features:
            flat = df_zone[col].astype(np.float32).values.reshape(-1, 1)
            if np.any(np.isnan(flat)) or np.any(np.isinf(flat)):
                print(f"Warning: NaN or Inf in column {col} for zone {zone_id}")
                flat = np.nan_to_num(flat, nan=0.0, posinf=0.0, neginf=0.0)
            if np.var(flat) < 1e-6:
                flat += np.random.normal(0, 1e-6, flat.shape)
            scaler = StandardScaler()
            df_zone[col] = scaler.fit_transform(flat).flatten()
            zone_scalers[col] = scaler

        dataset = ZoneDataset(df_zone, valid_features, target, SEQ_LEN)

        train_size = int(len(dataset) * 0.8)
        if train_size == 0 or len(dataset) - train_size == 0:
            print(f"Skipping zone {zone_id}: empty train/val set")
            continue

        train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

        model = LSTM_Model(input_size=len(valid_features)).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, factor=0.5)
        loss_fn = nn.HuberLoss()

        best_val_loss = float('inf')
        epochs_no_improve = 0

        all_train_preds, all_train_targets = [], []
        all_val_preds, all_val_targets = [], []

        for epoch in range(EPOCHS):
            model.train()
            train_loss = 0
            for xb, yb in train_loader:
                xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
                optimizer.zero_grad()
                with torch.autocast(device_type='cuda', dtype=torch.float16):
                    pred = model(xb)
                    loss = loss_fn(pred, yb)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                train_loss += loss.item()

                all_train_preds.append(pred.detach().cpu().numpy())
                all_train_targets.append(yb.detach().cpu().numpy())

            train_loss /= len(train_loader)


            model.eval()
            val_loss = 0
            with torch.no_grad():
                for xb, yb in val_loader:
                    xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
                    with torch.autocast(device_type='cuda', dtype=torch.float16):
                        pred = model(xb)
                        val_loss += loss_fn(pred, yb).item()
                        all_val_preds.append(pred.detach().cpu().numpy())
                        all_val_targets.append(yb.detach().cpu().numpy())

                val_loss /= len(val_loader)

            scheduler.step(val_loss)
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_no_improve = 0
                joblib.dump(model, os.path.join(MODEL_SAVE_DIR, f'zone_{zone_id}_model.pkl'))
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= PATIENCE:
                    print(f"Early stopping for zone {zone_id} at epoch {epoch}")
                    break

        # Inverse transform losses for interpretability
        train_preds_unscaled = np.expm1(scalers[target].inverse_transform(np.concatenate(all_train_preds).reshape(-1, 1)))
        train_targets_unscaled = np.expm1(scalers[target].inverse_transform(np.concatenate(all_train_targets).reshape(-1, 1)))
        val_preds_unscaled = np.expm1(scalers[target].inverse_transform(np.concatenate(all_val_preds).reshape(-1, 1)))
        val_targets_unscaled = np.expm1(scalers[target].inverse_transform(np.concatenate(all_val_targets).reshape(-1, 1)))

        train_rmse = np.sqrt(mean_squared_error(train_targets_unscaled, train_preds_unscaled))
        val_rmse = np.sqrt(mean_squared_error(val_targets_unscaled, val_preds_unscaled))

        print(f'Trained zone {zone_id}, Train Loss: {train_rmse:.4f}, Val Loss: {val_rmse:.4f}')
        plt.figure(figsize=(12, 6))
        plt.plot(val_targets_unscaled[:100], label='Actual')
        plt.plot(val_preds_unscaled[:100], label='Predicted')
        plt.legend()
        plt.title(f'Zone {zone_id} - Validation Forecast (First 100)')
        plt.xlabel("Timestep")
        plt.ylabel("Volume")
        plt.tight_layout()
        plot_path = os.path.join(MODEL_SAVE_DIR, f'zone_{zone_id}_forecast_plot.png')
        plt.savefig(plot_path)
        plt.close()

        # Single append to logs
        logs.append({
            'zone_id': zone_id,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_loss_unscaled': train_rmse,
            'val_loss_unscaled': val_rmse
        })

        with open(os.path.join(MODEL_SAVE_DIR, f'zone_{zone_id}_scalers.pkl'), 'wb') as f:
            pickle.dump(zone_scalers, f)

        with open(os.path.join(MODEL_SAVE_DIR, f'zone_{zone_id}_features.pkl'), 'wb') as f:
            pickle.dump(valid_features, f)

        del df_zone, dataset, train_loader, val_loader, model
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    except Exception as e:
        print(f"Failed for zone {zone_id}: {e}")

log_df = pd.DataFrame(logs)
log_df.to_csv(os.path.join(MODEL_SAVE_DIR, 'training_log.csv'), index=False)

Training per zone:   0%|          | 1/275 [00:06<31:53,  6.98s/it]

Early stopping for zone 1000 at epoch 23
Trained zone 1000, Train Loss: 43.5184, Val Loss: 11.5826


Training per zone:   1%|          | 2/275 [00:13<29:54,  6.57s/it]

Early stopping for zone 1009 at epoch 22
Trained zone 1009, Train Loss: 60.0579, Val Loss: 15.9390


Training per zone:   1%|          | 3/275 [00:20<30:50,  6.81s/it]

...

Early stopping for zone 991 at epoch 19
Trained zone 991, Train Loss: 4.9363, Val Loss: 1.7165


Training per zone: 100%|█████████▉| 274/275 [31:58<00:07,  7.71s/it]

Early stopping for zone 996 at epoch 23
Trained zone 996, Train Loss: 66.7169, Val Loss: 20.8717


Training per zone: 100%|██████████| 275/275 [32:04<00:00,  7.00s/it]

Early stopping for zone 998 at epoch 17
Trained zone 998, Train Loss: 19.9416, Val Loss: 6.8628


In [17]:
with open(os.path.join(MODEL_SAVE_DIR, 'global_volume_scaler.pkl'), 'wb') as f:
    pickle.dump(scalers['volume'], f)